In [768]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import bs4
import math

#import fpl_functions as fpl
pd.options.mode.chained_assignment = None
sns.set_theme()

In [769]:
#import FPL data

players22_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2022-23/gws/merged_gw.csv"
fixtures22_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2022-23/fixtures.csv"
teams22_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2022-23/teams.csv"

players23_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/gws/merged_gw.csv"
fixtures23_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/fixtures.csv"
teams23_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/teams.csv"

players24_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/merged_gw.csv"
fixtures24_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/fixtures.csv"
teams24_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/teams.csv"

players22 = pd.read_csv(players22_url)
fixtures22 = pd.read_csv(fixtures22_url)
teams22 = pd.read_csv(teams22_url)

players23 = pd.read_csv(players23_url)
fixtures23 = pd.read_csv(fixtures23_url)
teams23 = pd.read_csv(teams23_url)

players24 = pd.read_csv(players24_url)
fixtures24 = pd.read_csv(fixtures24_url)
teams24 = pd.read_csv(teams24_url)

### Get team market value from Transfermarkt

In [770]:
def get_team_market_value(year):
    url = 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id={}'.format(str(year))
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0'
    }
    req = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(req.text, 'html.parser')
    table = soup.find('table', class_ = 'items')
    titles = table.find_all('th')
    titles_table_headers = [title.text for title in titles]
    df = pd.DataFrame(columns = titles_table_headers)
    col_data = table.find_all('tr')

    for row in col_data[1:]:
        row_data = row.find_all('td')
        individual_row_data = [data.text for data in row_data]
        length = len(df)
        df.loc[length] = individual_row_data
    df.drop(0, axis=0, inplace=True)
    df.rename(columns={'ø age': 'avg age', 'ø market value': 'avg market value'}, inplace=True)
    df['avg market value'] = df['avg market value'].str.replace('€', '')
    df['avg market value'] = df['avg market value'].apply(lambda x: float(x.replace('bn', '')) if 'bn' in x else float(x.replace('m', '')))
    df['Total market value'] = df['Total market value'].str.replace('€', '')
    df['Total market value'] = df['Total market value'].apply(lambda x: float(x.replace('bn', '')) if 'bn' in x else float(x.replace('m', ''))/1000)
    df[['avg market value', 'Total market value']] = df[['avg market value', 'Total market value']].apply(pd.to_numeric)
    return df

In [771]:
transfermarkt22 = get_team_market_value(2022)
transfermarkt23 = get_team_market_value(2023)
transfermarkt24 = get_team_market_value(2024)

In [772]:
#wrangle team names in transfermarkt dataframes to match FPL data

def clean_tm_data(df):
    #remove pre- and su- fixes
    df['name'] = df['name'].str.replace(' FC', '')
    df['name'] = df['name'].str.replace('AFC ', '')
    df['name'] = df['name'].str.replace(' United', '')
    df['name'] = df['name'].str.replace(' Town', '')
    df['name'] = df['name'].str.replace(' & Hove Albion', '')
    #fix specific cases
    df['name'] = df['name'].str.replace('Manchester City', 'Man City')
    df['name'] = df['name'].str.replace('Manchester', 'Man Utd')
    df['name'] = df['name'].str.replace('Leicester City', 'Leicester')
    df['name'] = df['name'].str.replace('Sheffield', 'Sheffield Utd')
    df['name'] = df['name'].str.replace('Tottenham Hotspur', 'Spurs')
    df['name'] = df['name'].str.replace('Wolverhampton Wanderers', 'Wolves')
    df['name'] = df['name'].str.replace('Nottingham Forest', "Nott'm Forest")
    df['name'] = df['name'].str.strip()
    return df

In [773]:
transfermarkt22 = clean_tm_data(transfermarkt22)
transfermarkt23 = clean_tm_data(transfermarkt23)
transfermarkt24 = clean_tm_data(transfermarkt24)

In [774]:
#join transfermarkt data to FPL data
teams22 = teams22.merge(transfermarkt22[['name', 'avg age', 'Total market value']], on='name', how='left')
teams23 = teams23.merge(transfermarkt23[['name', 'avg age', 'Total market value']], on='name', how='left')
teams24 = teams24.merge(transfermarkt24[['name', 'avg age', 'Total market value']], on='name', how='left')

### Prepare Fixture Data

In [775]:
def prep_fixture_data(fixtures, teams):

    fixtures1 = fixtures.copy()
    fixtures1.drop(['code', 'finished_provisional', 'id', 'kickoff_time', 'minutes', 'provisional_start_time', 
                    'stats', 'team_h_difficulty', 'team_a_difficulty', 'pulse_id', 'team_h_score', 'team_a_score'], axis=1, inplace=True)
    
    away = fixtures1.copy()
    home = fixtures1.copy()

    home = home[['event', 'finished', 'started', 'team_h', 'team_a']]
    home.rename(columns={'team_h': 'team', 'team_a': 'opponent_team'}, inplace=True)
    home['was_home'] = 1

    away.rename(columns={'team_a': 'team', 'team_h': 'opponent_team'}, inplace=True)
    away['was_home'] = 0
    combined = pd.concat([home, away])
    combined['team'] = combined['team'].map(teams.set_index('id').name)
    combined['opponent_team'] = combined['opponent_team'].map(teams.set_index('id').name)
    combined.rename(columns={'event': 'GW'}, inplace=True)
    return combined

In [776]:
fixtures24_prepped = prep_fixture_data(fixtures24, teams24)

### Wrangle Player Data

In [777]:
#rename Dominic Solanke in 24/25 data
players24.loc[players24['name'] == 'Dominic Solanke-Mitchell', 'name'] = 'Dominic Solanke'

In [778]:
def prepare_player_data(players, teams):
    #join player's team market value
    players = players.merge(teams[['name', 'Total market value']].rename(columns={'name': 'team'}), how='left', on='team')
    
    #map opponent's team, join opponent's team market value
    players['opponent_team'] = players['opponent_team'].map(teams.set_index('id').name)
    players = players.merge(teams[['name', 'Total market value']].rename(columns={'name': 'opponent_team', 'Total market value': 'OPP Total market value'}), how='left', on='opponent_team')
    
    #drop columns
    players.drop(['round', 'xP', 'creativity', 'element', 'fixture', 'ict_index', 'influence', 'selected', 'threat', 'transfers_balance', 
                             'transfers_in', 'transfers_out', 'team_a_score', 'team_h_score', 'kickoff_time'], axis=1, inplace=True)
    
    #add goal involvements column
    players['goal_involvements'] = players['goals_scored'] + players['assists']
    
    #clean value column
    players['value'] = players['value']/10

    #boolean for whether player played match
    players['played_match'] = (players['minutes']>0).astype(int)

    players['was_home'] = players['was_home'].astype(int)

    #create expected goals and goal involvements vs actual goals and goal involvements columns
    players['g_vs_xg'] = players['goals_scored'] - players['expected_goals']
    players['gi_vs_xgi'] = players['goal_involvements'] - players['expected_goal_involvements']
    players['gc_vs_xgc'] = players['goals_conceded'] - players['expected_goals_conceded']

    #weighted columns?
    return players

In [779]:
players22 = prepare_player_data(players22, teams22)
players23 = prepare_player_data(players23, teams23)
players24 = prepare_player_data(players24, teams24)

In [780]:
#add boolean to indicate newly promoted teams
players22['vs_promoted'] = players22['opponent_team'].apply(lambda x: 1 if x in ['Fulham', 'Bournemouth', "Nott'm Forest"] else 0)
players23['vs_promoted'] = players23['opponent_team'].apply(lambda x: 1 if x in ['Burnley', 'Sheffield Utd', 'Luton'] else 0)
players24['vs_promoted'] = players24['opponent_team'].apply(lambda x: 1 if x in ['Ipswich', 'Leicester', 'Southampton'] else 0)

In [781]:
#combine 3 players dfs into 1

players22['season'] = 22
players22['total_GW'] = players22['GW']
players23['season'] = 23
players23['total_GW'] = players23['GW'] + 38
players24['season'] = 24
players24['total_GW'] = players24['GW'] + 76

players = pd.concat([players22, players23, players24])
players = players.reset_index(drop = True)

In [782]:
#drop players with 0 minutes
player_minutes = players.groupby('name')['minutes'].sum()
no_min = list(player_minutes[player_minutes == 0].index)
players.drop(players[players['name'].isin(no_min)].index, inplace=True)

In [783]:
#drop players below a certain threshold of minutes; 25th percentile
'''player_minutes2 = players.groupby('name')['minutes'].sum()
min_threshold = list(player_minutes2[player_minutes2 <= player_minutes2.quantile(0.25)].index)
players.drop(players[players['name'].isin(min_threshold)].index, inplace=True)'''

"player_minutes2 = players.groupby('name')['minutes'].sum()\nmin_threshold = list(player_minutes2[player_minutes2 <= player_minutes2.quantile(0.25)].index)\nplayers.drop(players[players['name'].isin(min_threshold)].index, inplace=True)"

In [784]:
#calculate xGc for each team, by season, up to a given gameweek
def get_xgc(df):
    return df.loc[df['position'] == 'GK'].groupby(['team', 'total_GW'])['expected_goals_conceded'].cumsum()

In [785]:
# need data to be lagged, since we are trying to predict future points based on past performance
# NOTE: some players, like Solanke, have different names in different seasons

agg_col = ['assists','bonus','bps','clean_sheets','expected_assists','expected_goal_involvements','expected_goals','expected_goals_conceded',
 'goals_conceded','goals_scored','minutes','saves','starts','total_points','OPP Total market value','goal_involvements','g_vs_xg',
 'gi_vs_xgi','gc_vs_xgc']

fixed_col = ['value', 'team', 'position', 'season', 'GW', 'Total market value', 'total_GW']

def lag_function(df, lag, num_GWs):
    #can I assume that every player's name is a unique identifier in the data set?
    df2 = df.copy()
    points = df2[['name', 'total_GW', 'opponent_team', 'OPP Total market value', 'was_home', 'total_points', 'vs_promoted']].copy()
    df2.drop(df2.loc[(df2['total_GW'] == num_GWs)].index, axis=0, inplace=True)
    points.rename(columns={'OPP Total market value': 'NW OPP Total market value', 'total_points': 'nw_points'}, inplace=True)
    df2['GW'] = df2['GW'] + 1
    df2['total_GW'] = df2['total_GW'] + 1

    #need total matches played to calculate lag window
    total_matches_played = pd.DataFrame(df2.groupby('name')['played_match'].sum().reset_index())
    total_matches_played.rename(columns={'played_match': 'total_matches_played'}, inplace=True)
    df2 = df2.merge(total_matches_played, how='left', on='name')

    #cumsum columns
    #cumsum_col = [i+'_cumsum' for i in agg_col]
    #df2[cumsum_col] = df2[agg_col].multiply(np.e**(-0.01 * (num_GWs + 1 - df2['total_GW'])), axis='index')
    #df2[cumsum_col] = df2.groupby('name')[cumsum_col].transform(lambda x: x.cumsum())

    #aggregation dict
    d1 = dict.fromkeys(agg_col+['yellow_cards', 'red_cards', 'played_match'], 'sum')
    d2 = dict.fromkeys(fixed_col, 'last')
    #d3 = dict.fromkeys(cumsum_col, 'last')
    #, 'total_points': 'var'
    d = {**d1, **d2}

    #initialize df to be returned
    full_data = pd.DataFrame()
    #first_gw = df2['total_GW'].min()
    i = lag #+ first_gw
    while i < num_GWs:
        loop_df = df2.copy()
        loop_df['rev_mp_cumsum'] = loop_df.groupby(['name'])['played_match'].transform(lambda x: x[::-1].cumsum()[::-1])
        # what happens when i > total_matches_played?
        cur_selection = loop_df.loc[(loop_df['total_GW'] <= i+1) & (loop_df['rev_mp_cumsum'] <= loop_df['total_matches_played']-i+1+lag) & (loop_df['played_match'] == 1)]

        temp = cur_selection.groupby(['name']).agg(d).reset_index()
        temp[agg_col] = temp[agg_col].div(temp['played_match'], axis=0)
        temp['total_GW'] = i
        temp = pd.merge(left=temp, right=points, left_on=['name', 'total_GW'], 
                  right_on=['name', 'total_GW'], how='left')
        full_data = pd.concat([full_data, temp])
        i += 1
    full_data.fillna(0, inplace=True)
    full_data.reset_index(drop=True, inplace=True)
    full_data.drop(full_data.loc[full_data['opponent_team'] == 0].index, inplace=True)
    return full_data

In [786]:
'''#find optimal lag window
test = []
for i in range(1, 10):
    test.append(lag_function(players, i, 85).corr()['nw_points'].sort_values(ascending=False)[1])'''

"#find optimal lag window\ntest = []\nfor i in range(1, 10):\n    test.append(lag_function(players, i, 85).corr()['nw_points'].sort_values(ascending=False)[1])"

In [787]:
players_lagged = lag_function(players, 6, 85)

In [788]:
players_lagged.loc[players_lagged['position'] == 'FWD'].corr()['nw_points'].sort_values(ascending=False)

nw_points                     1.000000
total_points                  0.577590
bps                           0.560892
goal_involvements             0.560748
goals_scored                  0.531610
bonus                         0.506379
gi_vs_xgi                     0.402059
minutes                       0.345256
expected_goal_involvements    0.338182
expected_goals                0.333658
g_vs_xg                       0.307419
assists                       0.296719
value                         0.296608
starts                        0.237665
played_match                  0.216848
expected_assists              0.211530
clean_sheets                  0.197359
season                        0.176034
total_GW                      0.174128
Total market value            0.163995
yellow_cards                  0.154034
expected_goals_conceded       0.147475
goals_conceded                0.138327
was_home                      0.066643
vs_promoted                   0.039865
GW                       

### ML Model

In [789]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from category_encoders.cat_boost import CatBoostEncoder

X = players_lagged.drop('nw_points', axis=1).reset_index(drop=True)
y = players_lagged['nw_points'].reset_index(drop=True)

cat_col = ['position', 'team', 'opponent_team']
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_encoded = enc.fit_transform(X[cat_col])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(cat_col)).reset_index(drop=True)
cat_enc = CatBoostEncoder()
cat_encoded = cat_enc.fit_transform(X['name'], y)
cat_enc_df = pd.DataFrame(cat_encoded, columns=cat_enc.get_feature_names_out('name')).reset_index(drop=True)

df_encoded = pd.concat([X, one_hot_df, cat_enc_df], axis=1)
df_encoded.drop(cat_col+['name'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df_encoded, y, test_size=0.2, random_state=442)

In [790]:
from sklearn.preprocessing import StandardScaler

std_scaler_train = StandardScaler()
std_scaler_test = StandardScaler()

X_train_s = std_scaler_train.fit_transform(X_train)
X_test_s = std_scaler_train.fit_transform(X_test)

In [ ]:
#Tuning
'''import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.05, 0.01],
    'subsample': [0.5, 0.75, 1],
    'colsample_bytree': [0.5, 0.75, 1],
    'n_estimators': [500, 1000, 1500],
    'lambda': [1, 3, 5]
}

reg = xgb.XGBRegressor()
grid_search = GridSearchCV(reg, param_grid, cv=5, scoring='neg_mean_absolute_error', verbose=3)
grid_search.fit(X_train, y_train)
print("Best score: ", grid_search.best_score_)
print("Best set of hyperparameters: ", grid_search.best_params_)'''

In [791]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

reg = xgb.XGBRegressor()
reg.fit(X_train_s, y_train)
pred = reg.predict(X_test_s)
rmse = np.sqrt(MSE(y_test, pred))
mae = MAE(y_test, pred)
print("RMSE : % f" %(rmse))
print("MAE : % f" %(mae))

RMSE :  2.830691
MAE :  1.839191


In [792]:
comparison = pd.concat([y_test.reset_index(), pd.Series(pred, name='predictions')], axis=1)
comparison['diff'] = abs(comparison['nw_points'] - comparison['predictions'])

In [793]:
comparison.loc[comparison['nw_points'] > 15].sort_values(by='diff', ascending=True).head(10)

,index,nw_points,predictions,diff
1576,7786,17.0,8.125255,8.874745
80,9732,16.0,6.850471,9.149529
1542,7959,16.0,6.625343,9.374657
985,4921,16.0,5.827752,10.172248
678,2224,19.0,8.675396,10.324604
138,8012,16.0,3.494586,12.505414
1285,8933,17.0,4.474715,12.525285
347,8909,18.0,5.217658,12.782342
1144,5973,16.0,2.333122,13.666878
669,6549,21.0,7.311488,13.688512


In [794]:
feature_importances = pd.concat([pd.Series(X_train.columns, name='Features'), pd.Series(reg.feature_importances_, name='Importance')], axis=1)
feature_importances.sort_values(by='Importance', ascending=False).tail(20)

,Features,Importance
11,saves,0.008032
4,expected_assists,0.007601
39,team_Chelsea,0.007302
19,yellow_cards,0.007080
77,opponent_team_Wolves,0.006934
40,team_Crystal Palace,0.006885
30,position_DEF,0.006825
70,opponent_team_Man Utd,0.006452
64,opponent_team_Ipswich,0.006362
69,opponent_team_Man City,0.006054


### Predict Future Weeks Points

In [795]:
def create_prediction_data(df, fixtures, teams, gw, lag):
    next_gw = df.loc[df['total_GW'] == gw].copy()
    next_gw['GW'] = next_gw['GW'] + 1
    next_gw['total_GW'] = next_gw['total_GW'] + 1
    next_gw.drop(['opponent_team', 'OPP Total market value', 'was_home'], axis=1, inplace=True)
    save_col = ['name', 'position', 'team', 'GW', 'total_GW', 'season', 'Total market value']
    clear_col = [i for i in next_gw.columns if i not in save_col]
    for col in clear_col:
        next_gw[col].values[:] = 0
    next_gw = pd.merge(left=next_gw, right=fixtures[['team', 'GW', 'opponent_team', 'was_home']], left_on=['team', 'GW'], right_on=['team', 'GW'], how='left')
    #next_gw = next_gw.merge(teams[['name', 'Total market value']].rename(columns={'name': 'team'}), how='left', on='team')
    next_gw = next_gw.merge(teams[['name', 'Total market value']].rename(columns={'name': 'opponent_team', 'Total market value': 'OPP Total market value'}), how='left', on='opponent_team')

    next_gw['GW'] = next_gw['GW'] - gw + lag
    next_gw['total_GW'] = next_gw['total_GW'] - gw + lag

    recent = df.loc[df['total_GW'] >= gw - lag]

    recent['GW'] = recent['GW'] - gw + lag
    recent['total_GW'] = recent['total_GW'] - gw + lag

    combined = pd.concat([recent, next_gw])
    combined_lagged = lag_function(combined, lag, lag+2)

    combined_lagged['GW'] = combined_lagged['GW'] + gw - lag
    combined_lagged['total_GW'] = combined_lagged['total_GW'] + gw - lag
    return combined_lagged

In [796]:
nw_pred_data = create_prediction_data(players, fixtures24_prepped, teams24, 84, 6)

In [797]:
X1 = nw_pred_data.loc[nw_pred_data['total_GW'] == 85].drop('nw_points', axis=1).reset_index(drop=True)

cat_col = ['position', 'team', 'opponent_team']
encoder_col = enc.get_feature_names_out(cat_col)
#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_encoded = enc.transform(X1[cat_col])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder_col).reset_index(drop=True)
#cat_enc1 = CatBoostEncoder()
cat_encoded = cat_enc.transform(X1['name'])
cat_enc_df1 = pd.DataFrame(cat_encoded, columns=cat_enc.get_feature_names_out('name')).reset_index(drop=True)

df_encoded = pd.concat([X1, one_hot_df, cat_enc_df1], axis=1)
df_encoded.drop(cat_col+['name'], axis=1, inplace=True)

std_scaler = StandardScaler()

X1_scaled = std_scaler.fit_transform(df_encoded)

In [798]:
pred1 = reg.predict(X1_scaled)

In [799]:
pd.concat([nw_pred_data.loc[nw_pred_data['total_GW'] == 85]['name'].reset_index(drop=True), pd.Series(pred1, name='predictions')], axis=1).sort_values(by='predictions', ascending=False).head(25)

,name,predictions
326,Ollie Watkins,14.366976
75,Cole Palmer,13.311677
314,Nicolas Jackson,12.354697
300,Mohamed Salah,11.781507
257,Luis Díaz,11.113518
350,Robert Sánchez,10.541880
127,Facundo Buonanotte,10.168958
378,Son Heung-min,9.966156
121,Erling Haaland,9.753712
291,Michael Keane,9.185101


In [800]:
nw_data = pd.concat([nw_pred_data.loc[nw_pred_data['total_GW'] == 85][['name', 'position', 'value', 'opponent_team']].reset_index(drop=True), pd.Series(pred1, name='predictions')], axis=1).sort_values(by='predictions', ascending=False)
nw_data.head(10)
#nw_data.to_excel('gw9_predictions.xlsx')

,name,position,value,opponent_team,predictions
326,Ollie Watkins,FWD,9.1,Bournemouth,14.366976
75,Cole Palmer,MID,10.8,Newcastle,13.311677
314,Nicolas Jackson,FWD,7.9,Newcastle,12.354697
300,Mohamed Salah,MID,12.6,Arsenal,11.781507
257,Luis Díaz,MID,8.0,Arsenal,11.113518
350,Robert Sánchez,GK,4.7,Newcastle,10.541880
127,Facundo Buonanotte,MID,5.0,Nott'm Forest,10.168958
378,Son Heung-min,MID,9.8,Crystal Palace,9.966156
121,Erling Haaland,FWD,15.4,Southampton,9.753712
291,Michael Keane,DEF,4.1,Fulham,9.185101


In [801]:
# try pulp optimizer
import pulp
metric = 'predictions'

x = pulp.LpVariable.dict("player", range(0, len(nw_data)),
                        0,1, cat=pulp.LpInteger)
prob = pulp.LpProblem("FantasyFootball", pulp.LpMaximize)
prob += pulp.lpSum(nw_data[metric].iloc[i] * x[i] for i in range(0, len(nw_data)))
prob += sum(x[i] for i in range(0, len(nw_data))) ==  11

prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'GK') == 1
prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'DEF') >= 3
prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'DEF') <= 5
 
prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'MID') >= 3
prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'MID') <= 5
 
prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'FWD') >= 1
prob  += sum(x[i] for i in range(0, len(nw_data)) if nw_data["position"].iloc[i] == 'FWD') <= 3

prob += sum(x[i] * nw_data["value"].iloc[i] for i in range(0, len(nw_data))) <= 83.3

print(prob.solve()) 
print(pulp.LpStatus[prob.status])
print(pulp.value(prob.objective))

1
Optimal
117.60606861114502


In [802]:
for i in range(0, len(nw_data)):
    if pulp.value(x[i]) == 1:
        print(nw_data['name'].iloc[i])

Ollie Watkins
Cole Palmer
Nicolas Jackson
Mohamed Salah
Luis Díaz
Robert Sánchez
Facundo Buonanotte
Son Heung-min
Michael Keane
James Justin
Ashley Young
